<a href="https://colab.research.google.com/github/squintero94/Pens_Ice_Time_Project/blob/main/xG_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

In [2]:
# Import our input dataset
pens_df = pd.read_csv('penguins_ice_time.csv')
pens_df.head()

,Date,Player,Opp,Shift_Count,Total_TOI,EV_TOI,PP_TOI,SH_TOI,Lineup,Score_For,...,Faceoff_Won,Faceoff_Loss,Faceoff_Pct,xG,xG/60,Corsi_%,Fenwick_%,Rebounds_Created_Above_x,On-Ice Goal Dif,On-Ice xG Dif
0,11/23/24,Rakell,UTA,22,17.85,14.25,3.60,0.0,12-6,1,...,0,0,NaN,0.2,0.73,71.8,65.5,-0.1,-1,0.8
1,11/23/24,Malkin,UTA,21,18.85,14.60,4.25,0.0,12-6,1,...,4,6,40.0,0.4,1.14,58.3,58.6,-0.2,1,0.5
2,11/23/24,Crosby,UTA,21,18.73,14.48,4.25,0.0,12-6,1,...,12,7,63.0,0.7,2.26,75.0,70.0,-0.4,-1,0.9
3,11/22/24,Rakell,WPG,23,20.48,16.68,3.80,0.0,12-6,1,...,0,1,0.0,0.4,1.04,57.8,51.5,-0.2,-1,-1.1
4,11/22/24,Malkin,WPG,20,17.83,13.85,3.98,0.0,12-6,1,...,7,10,41.0,0.5,1.78,56.8,41.7,0.7,-1,-1.1


In [25]:
# Encode (convert to dummy variables) the "Player" & "Opp" column
player_encoded = pd.get_dummies(pens_df['Player'], prefix="Player", dtype=int)
opp_encoded = pd.get_dummies(pens_df['Opp'], prefix="Opp", dtype=int)
lineup_encoded = pd.get_dummies(pens_df['Lineup'], prefix="Lineup", dtype=int)
result_encoded = pd.get_dummies(pens_df['Result'], prefix="Lineup", dtype=int)

In [9]:
player_encoded.head()

,Player_Crosby,Player_Malkin,Player_Rakell
0,0,0,1
1,0,1,0
2,1,0,0
3,0,0,1
4,0,1,0


In [10]:
opp_encoded.head()

,Opp_ANA,Opp_BOS,Opp_BUF,Opp_CAR,Opp_CBJ,Opp_CGY,Opp_COL,Opp_DAL,Opp_DET,Opp_EDM,...,Opp_NYI,Opp_NYR,Opp_OTT,Opp_SJS,Opp_TBL,Opp_TOR,Opp_UTA,Opp_VAN,Opp_WPG,Opp_WSH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
lineup_encoded.head()

,Lineup_12-6
0,1
1,1
2,1
3,1
4,1


In [27]:
result_encoded.head()

,Lineup_L,Lineup_OTL,Lineup_OTW,Lineup_SOL,Lineup_SOW,Lineup_W
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [28]:
# Concatenate the Pens DataFrame with the "Player" & "Opp" encoded dummies
new_pens_df = pd.concat([pens_df, player_encoded, opp_encoded, lineup_encoded, result_encoded], axis=1)

In [29]:
# Define the target variable 'xG' and features
y = new_pens_df["xG"].values
X = new_pens_df.drop(columns=["xG", "Date", "Player", "Opp", "Lineup", "Result"]).values

In [30]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [31]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
# Train a regression model
model = RandomForestRegressor(random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [34]:
# Make predictions
y_pred = model.predict(X_test_scaled)

In [35]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.005483000000000035
R^2 Score: 0.9452994687131048
